In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/commonlit-evaluate-student-summaries/sample_submission.csv
/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv
/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv
/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv
/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv


# BERT + Multi-layer Perceptron! Do upvote - motivates me to write more such notebooks for novice!

Simple implementation for begineers! 

In [2]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertModel
from transformers import ElectraTokenizer, ElectraModel

from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm  # Import tqdm for the progress bar

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Importing data into Dataframe

In [4]:
data = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv')

Tokenizing text and converting to tensors

In [5]:
tokenizer = ElectraTokenizer.from_pretrained('google/electra-base-discriminator')

max_seq_length = 128 #Use 512 for better results (if using GPU)

def tokenize_text(text):
    return tokenizer.encode(text, add_special_tokens=True, max_length=max_seq_length, padding='max_length', truncation=True)

data['text_tokens'] = data['text'].apply(tokenize_text)

# Split the data into features and targets
X = data['text_tokens'].tolist()
y = data[['wording', 'content']].values

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train).to(device)
X_test = torch.tensor(X_test).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱  1 tokenizer = ElectraTokenizer.from_pretrained('google/electra-base-discriminator')           │
│    2                                                                                             │
│    3 max_seq_length = 128 #Use 512 for better results (if using GPU)                             │
│    4                                                                                             │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1809 in          │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   1806 │   │   │   )                                                                             │
│   1807 │   │                                                                                     │
│   1808 │   │   if all(full_file_name is None for full_file_name in resolved_vocab_files.values(  │
│ ❱ 1809 │   │   │   raise EnvironmentError(                                                       │
│   1810 │   │   │   │   f"Can't load tokenizer for '{pretrained_model_name_or_path}'. If you wer  │
│   1811 │   │   │   │   "'https://huggingface.co/models', make sure you don't have a local direc  │
│   1812 │   │   │   │   f"Otherwise, make sure '{pretrained_model_name_or_path}' is the correct   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
OSError: Can't load tokenizer for 'google/electra-base-discriminator'. If you were trying to load it from 
'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make 
sure 'google/electra-base-discriminator' is the correct path to a directory containing all relevant files for a 
ElectraTokenizer tokenizer.

Loaded Transformer model (Use either ELECTRA or BERT)

In [ ]:
# base_model = BertModel.from_pretrained('bert-base-uncased').to(device)
base_model = ElectraModel.from_pretrained('google/electra-base-discriminator').to(device)

Generate Embeddings (Use GPU to go brrrrrr...)

In [ ]:
def generate_bert_embeddings(text_tokens):
    embeddings_list = []
    with tqdm(total=len(text_tokens)) as pbar:
        with torch.no_grad():
            for tokens in text_tokens:
                outputs = base_model(tokens.unsqueeze(0))  # Unsqueeze to add batch dimension
                embeddings = outputs.last_hidden_state[:, 0, :]  # Extract embeddings for [CLS] token
                embeddings_list.append(embeddings)
                pbar.update(1)
        embeddings_tensor = torch.cat(embeddings_list, dim=0)
        return embeddings_tensor

X_train_embeddings = generate_bert_embeddings(X_train)
X_test_embeddings = generate_bert_embeddings(X_test)

Make a Regression model to predict Content Score and Wording Score - A simple MLP (Multi-layer Perceptron)

In [ ]:
class RegressionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
    
#Defining a MCRMSE
def mean_columnwise_rmse(y_pred, y_true):
    columnwise_rmse = torch.sqrt(torch.mean((y_pred - y_true)**2, dim=0))
    return torch.mean(columnwise_rmse)

Model Parameters

In [ ]:
input_dim = base_model.config.hidden_size
hidden_dim = 256  # Adjust the hidden layer dimension as needed
output_dim = 2  # Number of target variables
model = RegressionModel(input_dim, hidden_dim, output_dim).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10 # Run it for 100-150 epoch
batch_size = 32 # 64 or 128 if using GPU

train_dataset = TensorDataset(X_train_embeddings, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

train_loss1_list = []
train_loss2_list = []
test_loss1_list = []
test_loss2_list = []

Train the model

In [ ]:
for epoch in range(num_epochs):
    model.train()
    total_loss_wording = 0
    total_loss_content = 0
    total_loss = 0
    
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss_wording = criterion(outputs[:, 0], targets[:, 0])
        loss_content = criterion(outputs[:, 1], targets[:, 1])

        loss = (loss_wording + loss_content) / 2
        loss.backward()
        optimizer.step()
        
        total_loss_wording += loss_wording.item()
        total_loss_content += loss_content.item()

        total_loss += loss.item()
        
    train_loss1_list.append(total_loss_wording / len(train_loader))
    train_loss2_list.append(total_loss_content / len(train_loader))
        
    print(f'Epoch [{epoch+1}/{num_epochs}], Wording Loss: {total_loss_wording:.4f}, Content Loss: {total_loss_content:.4f}, Loss: {total_loss:.4f}')
    
    model.eval()
    with torch.no_grad():
        test_outputs = model(X_test_embeddings)
        test_loss1 = criterion(test_outputs[:, 0], y_test[:, 0])
        test_loss2 = criterion(test_outputs[:, 1], y_test[:, 1])
        
    test_loss1_list.append(test_loss1.item())
    test_loss2_list.append(test_loss2.item())
    
    print(f'Test Loss Wording: {test_loss1_list[-1]:.4f}, Test Loss Content: {test_loss2_list[-1]:.4f}')

Collect losses and save as CSV for visualisation

In [ ]:
loss_data = {
    'Epoch': list(range(1, num_epochs+1)),
    'Train_Loss_Wording': train_loss1_list,
    'Train_Loss_Content': train_loss2_list,
    'Test_Loss_Wording': test_loss1_list,
    'Test_Loss_Content': test_loss2_list
}

loss_df = pd.DataFrame(loss_data)
loss_df.to_csv('losses.csv', index=False)

Make submission ready!

In [ ]:
test_data = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv')
test_data['text_tokens'] = test_data['text'].apply(tokenize_text)
test_text_tokens = test_data['text_tokens'].tolist()
test_text_tokens = torch.tensor(test_text_tokens).to(device)

In [ ]:
#create embeddings for test data
test_data_embeddings = generate_bert_embeddings(test_text_tokens)

In [ ]:
#Predict scores for test data
model.eval()
with torch.no_grad():
    test_data_outputs = model(test_data_embeddings)

In [ ]:
#Create a submission file
submission_df = pd.DataFrame()
submission_df['student_id'] = test_data['student_id']
submission_df['content'] = test_data_outputs[:, 1].cpu()
submission_df['wording'] = test_data_outputs[:, 0].cpu()

In [ ]:
submission_df.to_csv('submission.csv', index=False)

# Thanks! Do Upvote!!!!

# Will be sharing a notebook with additional features and more complex regression neural network